In [2]:
import pandas as pd
from statsmodels.tsa.holtwinters import ExponentialSmoothing

# Load your orders.csv (adjust path if needed)
orders = pd.read_csv("../data/raw/orders.csv", parse_dates=["order_date", "delivery_date", "promised_delivery_date"])

# Create OTD flag (1 = on time, 0 = late)
orders["otd_flag"] = (orders["delivery_date"] <= orders["promised_delivery_date"]).astype(int)

# Aggregate to monthly OTD rate
orders["month"] = orders["order_date"].dt.to_period("M").dt.to_timestamp()
monthly_otd = orders.groupby("month")["otd_flag"].mean()

# Fit Holt-Winters seasonal model
model = ExponentialSmoothing(monthly_otd, trend="add", seasonal="add", seasonal_periods=12)
fit = model.fit()

# Forecast next 6 months
forecast = fit.forecast(6)

# Save forecast
df_forecast = pd.DataFrame({
    "month": forecast.index,
    "otd_forecast": forecast.values
})
df_forecast.to_csv("../data/raw/otd_forecast.csv", index=False)

print("✅ otd_forecast.csv created!")
print(df_forecast)


✅ otd_forecast.csv created!
       month  otd_forecast
0 2025-09-01      0.974870
1 2025-10-01      0.975817
2 2025-11-01      0.974913
3 2025-12-01      0.974655
4 2026-01-01      0.974762
5 2026-02-01      0.973987


c:\Users\Fyras\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
